# Tabla Y4-125

In [ ]:
#!pip install specdal spectrum

Extraer datos

In [37]:
import os
from specdal import Collection, Spectrum, read
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

datadir = "./Espectros_FORS_2/Y4"
archivos = list()
specs = list()
cont = 0
aux = 0
etiquetas = ['A1','A2','A3','A4','A5',
             'B1','B2','B3','B4','B5',
             'C1','C2','C3','C4','C5',
             'D1','D2','D3','D4','D5',
             'E1','E2','E3','E4','E5',
            ]
features = []
for f in os.listdir(datadir):
    data = pd.read_csv(datadir+"/"+f,delimiter='\t')
    
    title =str(f)[:-4]
    dev_x = data['Wavelength']
    dev_y = data[title]
    
    #plt.plot(dev_x, dev_y)
    df = pd.DataFrame({'x': dev_x, 'y': dev_y})
    archivos.append(f)
    tmp_arr = df.to_numpy()
    specs.append(tmp_arr)

    tmp = np.array(dev_y)
    features.append(tmp)
    aux+=1
    if aux <= 125 and aux%5==0:
        #plt.ylabel('Reflectance')
        #plt.xlabel('Wavelength')
        t = "Tabla Y4",etiquetas[cont]
        cont+=1
        #plt.title(t)
        #plt.show()
len(features)

125

# Bayes ingenuo

In [66]:
# Función auxiliar  Cuenta las diferencias entre dos listas y 
# muestra el índice donde son diferentes.
def show_differences(list1, list2):
  differences = {}
  for i in range(len(list1)):
    if list1[i] != list2[i]:
      differences[i] = [list1[i], list2[i]]

  return differences

In [67]:
#feature_names = data['feature_names']
feature_names = ['wavelength','reflectance']
labels = np.arange(125)

train, test, train_labels, test_labels = train_test_split(
   features,labels,test_size = 0.00001, random_state = 22
)

GNBclf = GaussianNB()

model = GNBclf.fit(train, train_labels)
print(len(model.classes_))
test=features
preds = GNBclf.predict(test)
print(preds)
print(show_differences(preds, labels))

124
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  70  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124]
{72: [70, 72]}
